In [2]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps
import FL_utils.FL_Corel as Corel

# restore saved variables
%store -r summary_M6_fl_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [3]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_M_six_month'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_M_six_month'].values

In [4]:
#### CART
depth = [1,2]
impurity = [0.001, 0.003]
split = [2,3,4]
cart_summary = advance.CART(KY_X, KY_Y, FL_X, FL_Y, depth, split, impurity, seed=816)

### EBM
estimators = [40, 60]
depth = [1]
learning_rate = [0.01]
holdout_split = [0.9]
ebm_summary = advance.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [5]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.6162432825418865, 0.6248487050058547)

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence>=1']
KY_Y = KY_stumps['recid_M_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence>=1']
FL_Y = FL_stumps['recid_M_six_month'].values

## columns 
cols = KY_X.columns

In [24]:
c_grid = {'C':[0.05, 0.07, 0.09]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [25]:
stumps_summary

{'best_auc': 0.575887238864957,
 'best_params': {'C': 0.09},
 'auc_diffs': 0.07467211947690156,
 'KY_score': 0.5984379303413238}

### RiskSLIM

In [26]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.09, random_state=816).fit(FL_X, FL_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

21

In [27]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [30]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'general2', 
                                max_coef = 20, 
                                max_coef_number =10, 
                                max_runtime=200, 
                                c=1e-4, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
09/17/19 @ 01:32 PM | 941 rows in lookup table
09/17/19 @ 01:32 PM | ------------------------------------------------------------
09/17/19 @ 01:32 PM | runnning initialization procedure
09/17/19 @ 01:32 PM | ------------------------------------------------------------
09/17/19 @ 01:32 PM | CPA produced 2 cuts
09/17/19 @ 01:32 PM | running naive rounding on 70 solutions
09/17/19 @ 01:32 PM | best objective value: 0.3647
09/17/19 @ 01:32 PM | rounding produced 1 integer solutions
09/17/19 @ 01:32 PM | best objective value is 0.4133
09/17/19 @ 01:32 PM | running sequential rounding on 70 solutions
09/17/19 @ 01:32 PM | best objective value: 0.3647
09/17/19 @ 01:32 PM | sequential rounding produced 5 integer solutions
09/17/19 @ 01:32 PM | best objective value: 0.3750
09/17/19 @ 01:32 PM | polishing 6 solutions
09/17/19 @ 01:32 PM | best objective value: 0.3750
09/17/19 @ 01:32 PM | polishing produced 5 integer solutions
09/17/19 @

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
09/17/19 @ 01:32 PM | adding 191 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3734.
Tried aggregator 1 time.
Reduced MIP has 23 rows, 46 columns, and 87 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [31]:
riskslim_summary['KY_auc']

[0.6078705108473821]

### Corel

In [3]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence>=1']
KY_Y = KY_stumps['recid_M_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence>=1']
FL_Y = FL_stumps['recid_M_six_month'].values

In [4]:
KY_score, FL_score = Corel.corel_cv(KY_X, KY_Y, FL_X, FL_Y, max_card=2, c=1e-3, seed=816)

In [16]:
#### save results
summary_M6_fl_inter_model = {"CART": cart_summary,
                                    "EBM": ebm_summary,
                                    "Stumps": stumps_summary,
                                    "RiskSLIM": riskslim_summary, 
                                   "Corel": KY_score}
%store summary_M6_fl_inter_model

Stored 'summary_general2_fl_inter_model' (dict)


In [33]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Six Month\\"
results = [["Misdemeanor",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_auc'][0], 3), 
            round(KY_score, 3)]]
with open(path + 'Six Month FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)